In [3]:
## Initialization and imports
import pandas as pd 
import numpy as np

## Changing directory to top folder (All programs run from top)
import os
os.chdir('/Users/nschumacher/docs/galvanize/smunch-user-food-analysis')

# ## Changing directory to top folder (All programs run from top)
# import os
# os.chdir('/home/jovyan/work/docs/galvanize/smunch-user-food-analysis/')

from db.python_db import run_sql_query

# from pyspark.ml.recommendation import ALS
# from pyspark.sql import functions as F

from matplotlib import pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format='retina'

## Set random seed
np.random.seed(seed=14)

In [4]:
## Selecting pre created table with rating info
df = run_sql_query("SELECT * from noah.cust_ratings")
df.sample(3)

,cust_id,meal_id,meal_name,category,restaurant_name,delivery_tm,meal_rating
14991,0030N00002LQqcxQAD,a050N00000zZfz8QAC,Fajita Veggies Quinoa Bowl,freakyfit,Chupenga,2018-05-22 09:00:00+00:00,5.0
25804,0030N00002LQqMKQA1,a050N00000zZg49QAC,Grilled Teriyaki Salmon Bento,livinglight,Hashi Izakaya & Japanese Kitchen,2017-08-31 09:00:00+00:00,4.0
9812,0030N00002iBdCJQA0,a050N00000zbFdZQAU,Rice Noodle Salad with Chicken Breast,livinglight,Dave B.,2019-02-06 11:20:00+00:00,4.5


In [5]:
df = df[['cust_id', 'meal_name', 'meal_rating']]
df.sample(3)

,cust_id,meal_name,meal_rating
38716,0030N00002LQpCqQAL,Bibimbap with Minced Beef,4.0
37601,0030N00002LQpGrQAL,Fajita Veggies Quinoa Bowl,5.0
58627,0030N00002LQpm9QAD,Lamb and Veggie Curry,3.0


In [104]:
## Pivoting the table to get in format for ALS
table = pd.pivot_table(df,
                       values='meal_rating',
                       index=['cust_id'],
                       columns=['meal_name'],
                       fill_value=0)
table.sample(4)

meal_name,'Amatriciana' Casarecce Pasta with Guanciale Bacon (Pork cheek),'Be Veggie' Tofu Burger in Brioche Bun,'Berliner Bowl' with Beef-Meatballs,'Caeser's Delight' Chicken Salad with Anchovies & Fresh Veggies in Herb Dressing,'Com Chay' Lemongrass Tofu with Pan-fried Sesame-Veggies,'Com Ga' Lemongrass Chicken with Pan-fried Sesame-Veggies,'New Classic' Beef Burger with Bacon in Brioche Bun,'Pad Lao' Rice Noodles with Chicken,'Pad Lao' Rice Noodles with Tofu,'Pasta Bella Bologna' Fusili in Bolognese Beef Sauce (750ml),...,Wild Rice Bowl with Salmon,Winter Bowl with Yoghurt Dressing (only the bread contains gluten),Wok Chicken in Cocos-Curry,Woked Vegetables & Udon Noodles,Wrap with Mixed Salad & Balsamic Dressing,Za'tar Turkey with Couscous Salad,Zucchini-Aubergine-Paprika Sandwich with Bulgur-Chickpea Salad,Zula Special Chicken Salad,​Beef Kofte & Vermicelli-Rice ​,​​Japanese Bowl with Salmon Ceviche
cust_id,,,,,,,,,,,,,,,,,,,,,
0030N00002LQpyFQAT,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
0030N00002LQpOrQAL,0,0.0,0.0,0.0,0,0,0.0,0,0,4.0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,3.0,0.0
0030N00002WyAloQAF,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
0030N00002LQqPGQA1,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,...,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [8]:
## Querry to get the avg meal rating for each meal
querry = '''
SELECT
    product_name as meal_name,
    AVG(rating_score) as avg_meal_rating,
    COUNT(rating_score) as rating_count
FROM 
    bi.executed_order_employee
WHERE
    order_type = 'single' and rating_score IS NOT NULL
GROUP BY
    product_name'''
avg = run_sql_query(querry)
avg.sample(4)

,meal_name,avg_meal_rating,rating_count
703,Chicken-Pumpkin Curry,4.217391,23
786,Cheese Spätzle with Roast Onions,3.859524,210
467,Egyptian Rice-Lentil-Macaroni 'Koshary',4.384615,13
733,Kimchee Tacos with Honey Beef and Spicy Chicken,3.657738,168


In [32]:
## Function to assign missing values to avg
def con_avg(x, avg):
    if x == 0:
        return avg
    return x
    
## For each column get avg value and assign it to missing info
for meal in table.columns:
    avg_val = avg.loc[avg['meal_name'] == meal, :].avg_meal_rating.values[0]
    table[meal] = table[meal].apply(con_avg, args=(avg_val,))
    
table.sample(2)

meal_name,'Amatriciana' Casarecce Pasta with Guanciale Bacon (Pork cheek),'Be Veggie' Tofu Burger in Brioche Bun,'Berliner Bowl' with Beef-Meatballs,'Caeser's Delight' Chicken Salad with Anchovies & Fresh Veggies in Herb Dressing,'Com Chay' Lemongrass Tofu with Pan-fried Sesame-Veggies,'Com Ga' Lemongrass Chicken with Pan-fried Sesame-Veggies,'New Classic' Beef Burger with Bacon in Brioche Bun,'Pad Lao' Rice Noodles with Chicken,'Pad Lao' Rice Noodles with Tofu,'Pasta Bella Bologna' Fusili in Bolognese Beef Sauce (750ml),...,Wild Rice Bowl with Salmon,Winter Bowl with Yoghurt Dressing (only the bread contains gluten),Wok Chicken in Cocos-Curry,Woked Vegetables & Udon Noodles,Wrap with Mixed Salad & Balsamic Dressing,Za'tar Turkey with Couscous Salad,Zucchini-Aubergine-Paprika Sandwich with Bulgur-Chickpea Salad,Zula Special Chicken Salad,​Beef Kofte & Vermicelli-Rice ​,​​Japanese Bowl with Salmon Ceviche
cust_id,,,,,,,,,,,,,,,,,,,,,
0030N00002LQpb8QAD,4.1,2.769231,4.192308,3.846154,3.2,3.333333,3.429293,3.818182,3.714286,3.648352,...,5.0,2.0,4.125,3.518519,3.190476,2.833333,3.524648,3.181818,3.364103,4.578947
0030N00002jGUngQAG,4.1,2.769231,4.192308,3.846154,3.2,3.333333,3.429293,3.818182,3.714286,3.648352,...,5.0,2.0,4.125,3.518519,3.190476,2.833333,3.524648,3.181818,3.364103,4.578947


In [95]:
from sklearn.decomposition import NMF

## Taking subset of data to perform NMF on
X = np.round(table.values, 3)
subX = X[0:300,20:25]
X

array([[4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       ...,
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579],
       [4.1  , 2.769, 4.192, ..., 3.182, 3.364, 4.579]])

In [100]:
## Creating NMF object
nmf= NMF(max_iter=100, n_components=5, solver='cd')

W = nmf.fit_transform(X)  ## W matrix: has n hidden user topics
H = nmf.components_          ## H matrix: has n hidden meal topics

In [101]:
np.round(W, 2)

array([[1.23, 0.04, 0.16, 0.07, 0.07],
       [1.23, 0.04, 0.09, 0.07, 0.07],
       [1.24, 0.02, 0.12, 0.08, 0.  ],
       ...,
       [1.24, 0.04, 0.09, 0.07, 0.07],
       [1.23, 0.04, 0.09, 0.06, 0.07],
       [1.24, 0.05, 0.07, 0.06, 0.06]])

In [102]:
np.round(H, 2)

array([[3.3 , 2.21, 3.37, ..., 2.56, 2.67, 3.69],
       [0.  , 0.  , 0.67, ..., 0.07, 1.09, 0.  ],
       [0.23, 0.2 , 0.  , ..., 0.05, 0.  , 0.24],
       [0.  , 0.19, 0.05, ..., 0.06, 0.08, 0.  ],
       [0.  , 0.15, 0.  , ..., 0.11, 0.2 , 0.  ]])

In [103]:
preds = np.dot(W, H)
np.round(preds, 2)

array([[4.1 , 2.77, 4.18, ..., 3.17, 3.35, 4.57],
       [4.1 , 2.77, 4.19, ..., 3.18, 3.36, 4.57],
       [4.11, 2.77, 4.18, ..., 3.18, 3.33, 4.59],
       ...,
       [4.1 , 2.77, 4.19, ..., 3.18, 3.36, 4.58],
       [4.1 , 2.77, 4.19, ..., 3.18, 3.36, 4.58],
       [4.1 , 2.76, 4.2 , ..., 3.18, 3.38, 4.58]])